In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tnguy70/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tnguy70/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe();

In [5]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name']))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['cozy', 'entire', 'floor', 'of', 'brownstone']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['cozy', 'entire', 'floor', 'brownstone']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

/software/anaconda3/5.3.0/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [13]:
# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.136*"beautiful" + 0.084*"time" + 0.083*"location" + 0.077*"new" + '
  '0.066*"luxury" + 0.063*"good" + 0.055*"great" + 0.053*"amazing" + '
  '0.033*"step" + 0.030*"brand"'),
 (1,
  '0.229*"spacious" + 0.069*"side" + 0.046*"stay" + 0.041*"floor" + '
  '0.035*"hotel" + 0.035*"low" + 0.034*"luxurious" + 0.027*"loft" + '
  '0.022*"sleep" + 0.021*"east"'),
 (2,
  '0.171*"cozy" + 0.159*"modern" + 0.133*"home" + 0.073*"away" + '
  '0.041*"garden" + 0.035*"single" + 0.032*"block" + 0.032*"brownstone" + '
  '0.029*"friendly" + 0.026*"flat"'),
 (3,
  '0.209*"studio" + 0.090*"comfortable" + 0.073*"min" + 0.058*"minute" + '
  '0.053*"subway" + 0.037*"charming" + 0.037*"next" + 0.034*"cute" + '
  '0.028*"artist" + 0.024*"little"'),
 (4,
  '0.206*"bed" + 0.174*"sunny" + 0.107*"large" + 0.098*"train" + '
  '0.032*"neighborhood" + 0.030*"parking" + 0.027*"relax" + 0.024*"open" + '
  '0.019*"bushwick" + 0.019*"balcony"'),
 (5,
  '0.093*"bright" + 0.080*"renovate" + 0.075*"view" + 0.060*"huge"

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.86540353434641

Coherence Score:  0.502321066623062


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/software/anaconda3/5.3.0/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.305067  0.310848       1        1  10.344047
7      0.041221  0.056481       2        1  10.085335
4     -0.103903  0.020073       3        1  10.020020
5     -0.036987 -0.010421       4        1  10.006613
2     -0.129159 -0.010583       5        1  10.002121
1     -0.166502 -0.002230       6        1   9.999924
9      0.257016 -0.371345       7        1   9.955675
0      0.057162 -0.002661       8        1   9.915488
6     -0.136788  0.014993       9        1   9.853969
3     -0.087128 -0.005156      10        1   9.816801, topic_info=     Category           Freq         Term          Total  loglift  logprob
term                                                                      
10    Default  212173.000000         room  212173.000000  30.0000  30.0000
60    Default  187363.000000      private  187363.000000  29.0000  29.0000
24    Default  141846.000000      bedroom  141846.000000  28.0000  28.0000
7     Default  114649.000000    apartment  114649.000000  27.0000  27.0000
5     Default   62251.000000     spacious   62251.000000  26.0000  26.0000
6     Default   56013.000000       studio   56013.000000  25.0000  25.0000
40    Default   56116.000000          bed   56116.000000  24.0000  24.0000
38    Default   47614.000000        sunny   47614.000000  23.0000  23.0000
1     Default   46679.000000         cozy   46679.000000  22.0000  22.0000
76    Default   43372.000000       modern   43372.000000  21.0000  21.0000
55    Default   36656.000000    beautiful   36656.000000  20.0000  20.0000
51    Default   36271.000000         home   36271.000000  19.0000  19.0000
73    Default   35307.000000        heart   35307.000000  18.0000  18.0000
8     Default   29099.000000        large   29099.000000  17.0000  17.0000
224   Default   26786.000000        train   26786.000000  16.0000  16.0000
62    Default   25383.000000       bright   25383.000000  15.0000  15.0000
70    Default   24099.000000  comfortable   24099.000000  14.0000  14.0000
85    Default   23749.000000     bathroom   23749.000000  13.0000  13.0000
102   Default   22746.000000         time   22746.000000  12.0000  12.0000
66    Default   22535.000000     location   22535.000000  11.0000  11.0000
206   Default   21901.000000     renovate   21901.000000  10.0000  10.0000
101   Default   20913.000000        quiet   20913.000000   9.0000   9.0000
168   Default   20704.000000          new   20704.000000   8.0000   8.0000
112   Default   20463.000000         view   20463.000000   7.0000   7.0000
108   Default   20551.000000        clean   20551.000000   6.0000   6.0000
200   Default   19854.000000         away   19854.000000   5.0000   5.0000
178   Default   19763.000000        close   19763.000000   4.0000   4.0000
151   Default   19534.000000          min   19534.000000   3.0000   3.0000
69    Default   18943.000000      village   18943.000000   2.0000   2.0000
18    Default   18921.000000         side   18921.000000   1.0000   1.0000
...       ...            ...          ...            ...      ...      ...
151   Topic10   19533.488281          min   19534.390625   2.3210  -2.6165
311   Topic10   15548.715820       minute   15549.619141   2.3210  -2.8447
318   Topic10   14193.916992       subway   14194.819336   2.3210  -2.9358
258   Topic10    9910.986328     charming    9911.889648   2.3210  -3.2950
223   Topic10    9834.641602         next    9835.544922   2.3210  -3.3027
190   Topic10    9204.869141         cute    9205.772461   2.3210  -3.3689
50    Topic10    7570.689453       artist    7571.592285   2.3210  -3.5644
88    Topic10    6288.586426       little    6289.489258   2.3209  -3.7499
238   Topic10    5622.962402     stunning    5623.865234   2.3209  -3.8618
97    Topic10    5295.940918       height    5296.843750   2.3209  -3.9217
280   Topic10    4228.839844         high    4229.742676   2.3209  -4.1467
780   Top